In [ ]:
import os
import numpy as np
from scipy.optimize import minimize
from simsopt.geo import SurfaceRZFourier
from simsopt.objectives import SquaredFlux
from simsopt.objectives import QuadraticPenalty
from simsopt.geo import curves_to_vtk
from simsopt.field import BiotSavart, Current, coils_via_symmetries
from simsopt.geo import (
    CurveLength, CurveCurveDistance,
    MeanSquaredCurvature, LpCurveCurvature, CurveCWSFourier, ArclengthVariation, CurveXYZFourier
)

import matplotlib.pyplot as plt

In [ ]:
wout = '../input.axiTorus_nfp3_QA_final'

ncoils = 1
order = 1 # order of dofs of cws curves
quadpoints = 300 #13 * order
ntheta = 50
nphi = 42
ext_via_normal_factor = 0.2565

In [ ]:
s = SurfaceRZFourier.from_vmec_input(wout, range="half period", ntheta=ntheta, nphi=nphi)
s_full = SurfaceRZFourier.from_vmec_input(wout, range="full torus", ntheta=ntheta, nphi=int(nphi*2*s.nfp))
# CREATE COIL WINDING SURFACE SURFACE
cws = SurfaceRZFourier.from_vmec_input(wout, range="half period", ntheta=ntheta, nphi=nphi)
cws_full = SurfaceRZFourier.from_vmec_input(wout, range="full torus", ntheta=ntheta, nphi=int(nphi*2*s.nfp))

cws.extend_via_normal(ext_via_normal_factor)
cws_full.extend_via_normal(ext_via_normal_factor)

# CREATE CURVES + COILS     
base_curves = []
for i in range(ncoils):
    curve_cws = CurveCWSFourier(
        mpol=cws.mpol,
        ntor=cws.ntor,
        idofs=cws.x,
        quadpoints=quadpoints,
        order=order,
        nfp=cws.nfp,
        stellsym=cws.stellsym,
    )
    angle = (i + 0.5)*(2 * np.pi)/((2) * s.nfp * ncoils)
    curve_dofs = np.zeros(len(curve_cws.get_dofs()),)
    curve_dofs[0] = 1
    curve_dofs[2*order+2] = 0
    curve_dofs[2*order+3] = angle
    curve_cws.set_dofs(curve_dofs)
    curve_cws.fix(0)
    curve_cws.fix(2*order+2)
    base_curves.append(curve_cws)
base_currents = [Current(1)*1e5 for i in range(ncoils)]

coils = coils_via_symmetries(base_curves, base_currents, s.nfp, True)

saddle_coil = CurveCWSFourier(mpol=cws.mpol, ntor=cws.ntor, idofs=cws.x, quadpoints=quadpoints, order=1, nfp=cws.nfp, stellsym=cws.stellsym)
print(saddle_coil.num_dofs())
saddle_coil.set_dofs(np.array([0. , 1, 0.5, 0, 0. , 1, 0, 0.4]))


# fig = plt.figure()
# ax = plt.axes(projection='3d')
ax = s_full.plot(color="blue", close=True, show=False, engine="plotly", opacity=0.5)
ax.update_layout(showlegend=False)
cws_full.plot(ax=ax, color="lightblue", show=False, engine="plotly", close=True, opacity=0.5)
ax.update_layout(showlegend=False)
saddle_coil.plot(ax=ax, color="red", show=False, engine="plotly", close=True)
for coil in coils:
    coil.plot(ax=ax, color="green", show=False, engine="plotly", close=True)
# set box size
ax.update_layout(autosize=False, width=800, height=800)
# remove all bars 
ax.update_layout(showlegend=False)
ax.show()

In [ ]:
OUT_DIR = "./saddlecoils/"
os.makedirs(OUT_DIR, exist_ok=True)

bs = BiotSavart(coils)
bs.set_points(s_full.gamma().reshape((-1, 3)))
curves = [c.curve for c in coils]
curves_to_vtk(curves, OUT_DIR + "curves_init")
curves_to_vtk(base_curves, OUT_DIR + "base_curves_init")
pointData = {"B_N": np.sum(bs.B().reshape((int(nphi*2*s_full.nfp), ntheta, 3)) * s_full.unitnormal(), axis=2)[:, :, None]}
s_full.to_vtk(OUT_DIR + "surf_init", extra_data=pointData)
cws_full.to_vtk(OUT_DIR + "cws_init")


In [ ]:

mpol=1
ntor=0
num_dofs = 2*(mpol+1)*(2*ntor+1) - ntor - (ntor+1)
ntheta = 500
nphi = 500
wout = '../input.axiTorus_nfp3_QA_final'

# CREATE FLUX SURFACE (BOUNDARY)
s = SurfaceRZFourier.from_vmec_input(wout, range="half period", ntheta=ntheta, nphi=nphi)
s_full = SurfaceRZFourier.from_vmec_input(wout, range="full torus", ntheta=ntheta, nphi=int(nphi*2*s.nfp))
# CREATE COIL WINDING SURFACE
cws_full = SurfaceRZFourier.from_nphi_ntheta(nphi, ntheta, "half period", s.nfp)
surface = SurfaceRZFourier.from_nphi_ntheta(int(nphi*2*s.nfp), ntheta, "full torus", s.nfp)

R = s.get_rc(0, 0)
minor_radius_factor_cws = 1 + 0.2595/s.get_zs(1, 0)
surface.set_dofs([R, s.get_zs(1, 0)*minor_radius_factor_cws, s.get_zs(1, 0)*minor_radius_factor_cws])
cws_full.set_dofs([R, s.get_zs(1, 0)*minor_radius_factor_cws, s.get_zs(1, 0)*minor_radius_factor_cws])

curve = CurveCWSFourier(surface.mpol, surface.ntor, surface.x, 150, 0, surface.nfp, surface.stellsym)
curve.set_dofs([6, 0, 1, 0])

gammadash = curve.gammadash()
x_d = gammadash[:, 0]/100
y_d = gammadash[:, 1]/100
z_d = gammadash[:, 2]/100

# surface.plot(show=True, close=True, engine="plotly")
curve.plot(show=False, close=True, engine="plotly")


# curve.plot(ax=ax, show=False, plot_derivative=False, close=True, engine="plotly")
# surface.plot(ax=ax, show=False, alpha=0.2, close=True)
#ax.plot(x_d, y_d, z_d)